# Whitened Bias-Invariant GLRT Localization

This notebook demonstrates the **Whitened Bias-Invariant Iterative GLRT** approach for transmitter localization:
1. Load processed power measurements from monitoring locations
2. Run the iterative GLRT algorithm to detect transmitters one by one
3. Visualize the detected transmitters and estimated powers

---

## Overview

**GLRT Approach:**
- Iteratively tests for the presence of a new transmitter in the residual.
- Uses **whitening** to handle correlated sensor noise.
- Uses **orthogonal projection** to be invariant to unknown additive bias (e.g., noise floor shifts).
- Provides a sparse list of detected transmitters.

---

In [ ]:
# Import necessary modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yaml
import sys
import time
from pathlib import Path

# Add parent directory to path
sys.path.insert(0, str(Path.cwd().parent))

from src.utils import load_slc_map, load_monitoring_locations, get_sensor_locations_array, load_transmitter_locations
from src.localization.likelihood import build_covariance_matrix
from src.sparse_reconstruction.propagation_matrix import compute_propagation_matrix
from src.sparse_reconstruction.reconstruction import dbm_to_linear, linear_to_dbm
from src.glrt.solver import solve_glrt
from src.visualization.spatial_plots import plot_transmit_power_map

print("✓ Modules imported successfully")

In [ ]:
# Configure which transmitter(s) to analyze
TRANSMITTERS = ['mario', 'guesthouse']
TRANSMITTER_COMMA = ",".join(TRANSMITTERS)
TRANSMITTER_UNDERSCORE = "_".join(TRANSMITTERS)

print(f"Analyzing transmitter(s): {TRANSMITTER_COMMA}")

#!python ../scripts/process_raw_data_to_monitoring.py --input-dir "C:/Users/serha/raw_data/stat_rot/stat/" --transmitter {TRANSMITTER_COMMA} --num-locations 10 --output-yaml ../config/monitoring_locations_{TRANSMITTER_UNDERSCORE}.yaml --output-data ../data/processed/{TRANSMITTER_UNDERSCORE}/ --dedup-threshold 5.0

In [ ]:
# Set file paths
CUSTOM_YAML = f'../config/monitoring_locations_{TRANSMITTER_UNDERSCORE}.yaml'
DATA_DIR = Path(f'../data/processed/{TRANSMITTER_UNDERSCORE}/')

print(f"Configuration file: {CUSTOM_YAML}")
print(f"Data directory: {DATA_DIR}")

In [ ]:
# Load configuration
with open('../config/parameters.yaml', 'r') as f:
    config = yaml.safe_load(f)

print("Configuration loaded")

In [ ]:
# Load SLC map
print("Loading SLC map...")
map_data = load_slc_map(
    map_folder_dir="../",
    downsample_factor=config['spatial']['downsample_factor']
)

print(f"✓ Map loaded: shape {map_data['shape']}")

In [ ]:
# Load transmitter locations for visualization
print("Loading transmitter locations...")
all_tx_locations = load_transmitter_locations(
    config_path='../config/transmitter_locations.yaml',
    map_data=map_data
)

tx_locations = {name: all_tx_locations[name] for name in TRANSMITTERS if name in all_tx_locations}
print(f"✓ Loaded {len(tx_locations)} transmitter location(s) for display")

In [ ]:
# Load monitoring locations
print(f"Loading monitoring locations from {CUSTOM_YAML}...")
locations_config = load_monitoring_locations(
    config_path=CUSTOM_YAML,
    map_data=map_data
)
data_points = get_sensor_locations_array(locations_config)
print(f"✓ Loaded {len(data_points)} monitoring locations")

In [ ]:
# Load processed power measurements
print(f"Loading power measurements from {DATA_DIR}...")
observed_powers_dB = np.load(DATA_DIR / f"{TRANSMITTER_UNDERSCORE}_avg_powers.npy")

# Convert to linear scale (mW)
observed_powers_linear = dbm_to_linear(observed_powers_dB)

print(f"✓ Loaded power measurements (dB): {observed_powers_dB}")
print(f"  Linear scale (mW): {observed_powers_linear}")

## Run GLRT Algorithm

In [ ]:
# 1. Build Covariance Matrix
print("Building covariance matrix...")
cov_matrix = build_covariance_matrix(
    sensor_locations=data_points,
    sigma=config['localization']['std_deviation'],
    delta_c=config['localization']['correlation_coeff'],
    scale=config['spatial']['proxel_size']
)
print(f"✓ Covariance matrix shape: {cov_matrix.shape}")

# 2. Build Propagation Matrix
print("Building propagation matrix (this may take a moment)...")
A_model = compute_propagation_matrix(
    sensor_locations=data_points,
    map_shape=map_data['shape'],
    scale=config['spatial']['proxel_size'],
    np_exponent=config['localization']['path_loss_exponent'],
    vectorized=True,
    verbose=True
)
print(f"✓ Propagation matrix shape: {A_model.shape}")

In [ ]:
# 3. Run GLRT Solver
print("Running Whitened Bias-Invariant GLRT...")
start_time = time.time()

# Threshold: Controls sensitivity. Lower = more detections.
# Since score is bounded by M (number of sensors), threshold must be < M.
threshold = 4.0 

active_set, est_powers, est_bias, info = solve_glrt(
    observed_powers=observed_powers_linear,
    A_model=A_model,
    cov_matrix=cov_matrix,
    threshold=threshold,
    max_iter=10,
    verbose=True
)

elapsed_time = time.time() - start_time
print(f"\n✓ GLRT completed in {elapsed_time:.2f} seconds")
print(f"  Detected {len(active_set)} transmitters")
print(f"  Estimated Bias: {est_bias:.2e} mW ({linear_to_dbm(est_bias):.2f} dBm)")

In [ ]:
# 4. Visualize Results

# Convert sparse results to full map for visualization
height, width = map_data['shape']
glrt_map_linear = np.zeros((height, width))

for idx, power in zip(active_set, est_powers):
    row = idx // width
    col = idx % width
    glrt_map_linear[row, col] = power

# Convert to dBm (avoid log(0))
glrt_map_dbm = linear_to_dbm(glrt_map_linear)
# Set zeros to -inf or a low floor for visualization
glrt_map_dbm[glrt_map_linear <= 0] = -120.0

print("Visualizing GLRT results...")
fig, ax = plot_transmit_power_map(
    transmit_power_map=glrt_map_dbm,
    data_points=data_points,
    observed_powers=observed_powers_dB,
    UTM_lat=map_data['UTM_lat'],
    UTM_long=map_data['UTM_long'],
    band_name=f"GLRT Detection (Bias: {linear_to_dbm(est_bias):.1f} dBm)",
    transmitter_locations=tx_locations
)
plt.show()

In [ ]:
# Print detected locations
print("Detected Transmitters:")
print("--------------------------------------------------")
print(f"{'Rank':<5} {'Row':<5} {'Col':<5} {'Power (dBm)':<15} {'Power (mW)':<15}")
print("--------------------------------------------------")

# Sort by power descending
sorted_indices = np.argsort(est_powers)[::-1]

for rank, i in enumerate(sorted_indices):
    idx = active_set[i]
    power = est_powers[i]
    row = idx // width
    col = idx % width
    print(f"{rank+1:<5} {row:<5} {col:<5} {linear_to_dbm(power):<15.2f} {power:<15.2e}")